## Diego Hurtado // Suzaku
diego.hurtado@userena.cl \
Terza prueba de caracterización para FALCON @ ULS \
Agosto 25
Rehaciendo la caracterizacion

In [1]:
#Comienzo importando todo lo que ocupo
import numpy as np
import glob
import matplotlib.pyplot as plt
from pprint import pprint
import os, re, sys
import bqplot as bq

import astropy
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.nddata import CCDData
from astropy import units as u
import ccdproc as ccdp
from scipy.ndimage import gaussian_filter
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import uniform_filter, median_filter
from scipy.stats import norm
import matplotlib.mlab as mlab

In [ ]:
#Checkeo manual de header de las imagenes
dirname = './images/11-09/lights/' # Cambia esto al directorio donde están los respectivos lights
filename = ""

with fits.open(os.path.join(dirname, filename)) as hdul:
    hdul.info()
header = fits.getheader(os.path.join(dirname, filename))
pprint(header)

Filename: ./images/11-09/lights/00000001.NGC 1097.fit
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      43   (4788, 3190)   int16 (rescales to uint16)   
SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 4788 / length of data axis 1                          
NAXIS2  =                 3190 / length of data axis 2                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
BZERO   =                32768 / offset data range to that of unsigned short    
BSCALE  =                    1 / default scaling factor             

In [ ]:
#Renombro las imagenes en bruto para facilidad de lectura de achivos
def renamer(dirname):     #Valido FALCON por Keywords (OBJECT-FILTER-EXPTIMEs-IMAGETYP.fit)
    dirname = dirname
    hdu = 0
    filequan = 0 #Cuantificador de archivos
    for path in os.listdir(dirname):
        if os.path.isfile(os.path.join(dirname, path)):
            filequan += 1
    print(filequan)

#3 deep indents

    i = 0
    if filequan == 0:
        message = f'No files ({i}) to rename in {dirname}' #Control
    if filequan > 0:
        for fitsName in sorted(glob.glob(dirname+'*.fit')): #Esto deberia ciclar por los archivos en carpeta
            header = fits.getheader(fitsName, hdu)
            os.rename(fitsName,f"{dirname}{header.get("IMAGETYP")}-{header.get("FILTER")}-{header.get("OBJECT")}-{i}.fit")
            #print(f'{fitsName} renombrada') #Control
            i=i+1
        message = f'Renamed {i} file(s) in {dirname}' #Control
    return(message)


print(renamer('./images/11-09/lights/'))
print(renamer('./images/11-09/bias/'))
print(renamer('./images/11-09/darks/'))
print(renamer('./images/11-09/flats/'))

188
Renamed 188 file(s) in ./images/11-09/lights/
101
Renamed 101 file(s) in ./images/11-09/bias/
226
Renamed 226 file(s) in ./images/11-09/darks/
313
Renamed 313 file(s) in ./images/11-09/flats/


In [13]:
#Checking de headers
hdu = 0
fitsNames = []
dirname = './images/11-09/lights/'
keys=['OBJECT','EXPTIME','FILTER','FOCPOS','AIRMASS','CCD-TEMP']

#For para cada archivo fits en el directorio establecido (dir) printeamos su header con las keywords seleccionadas a mano
for fitsName in sorted(glob.glob(dirname+'*.fit')):
    pprint(fitsName.replace(dirname,'File'))
    header = fits.getheader(fitsName, hdu)
    outkeys = [(key, header.get(key)) for key in keys]
    for kwrd in outkeys:
        print( f"{kwrd[0]:<10} =  {kwrd[1]}")
    #Alinear string format
    print('\n')

'./images/11-09/lights\\Light Frame-1-Aldebaran-0.fit'
OBJECT     =  Aldebaran
EXPTIME    =  10.0
FILTER     =  1
FOCPOS     =  None
AIRMASS    =  3.18287287976892
CCD-TEMP   =  0.0


'./images/11-09/lights\\Light Frame-1-Aldebaran-1.fit'
OBJECT     =  Aldebaran
EXPTIME    =  5.0
FILTER     =  1
FOCPOS     =  None
AIRMASS    =  3.16167218548728
CCD-TEMP   =  0.0


'./images/11-09/lights\\Light Frame-1-Betelgeuse-2.fit'
OBJECT     =  Betelgeuse
EXPTIME    =  2.0
FILTER     =  1
FOCPOS     =  None
AIRMASS    =  9.02181520745142
CCD-TEMP   =  0.0


'./images/11-09/lights\\Light Frame-1-Betelgeuse-3.fit'
OBJECT     =  Betelgeuse
EXPTIME    =  5.0
FILTER     =  1
FOCPOS     =  None
AIRMASS    =  8.91311456629421
CCD-TEMP   =  0.0


'./images/11-09/lights\\Light Frame-1-NGC 104-4.fit'
OBJECT     =  NGC 104
EXPTIME    =  10.0
FILTER     =  1
FOCPOS     =  None
AIRMASS    =  1.34624893426626
CCD-TEMP   =  0.0


'./images/11-09/lights\\Light Frame-1-NGC 2070-5.fit'
OBJECT     =  NGC 2070
EXPTIM

In [15]:
usigma, lsigma = 2,2
with fits.open('./images/11_09/lights/Light Frame-11-NGC 253-161.fit') as hdul:
    data = hdul[0].data
    header = hdul[0].header
datamedian = np.median(data)
datastd = np.std(data)
plt.imshow(data, origin='lower', cmap='Blues', vmin=np.mean(data)-lsigma*np.std(data), vmax=np.mean(data)+usigma*np.std(data))
plt.tick_params(left = False, right = False, labelleft = False, labelbottom = False, bottom = False) 
plt.title('Light Frame-11-NGC 253-161.fit')
plt.show

FileNotFoundError: [Errno 2] No such file or directory: './images/11_09/lights/Light Frame-11-NGC 253-161.fit'

La imagen de arriba muestra cómo resulta una imagen al cortarse la alimentación en plena exposición

In [ ]:
#Checkeo simultáneo de imágenes de cada imagen, se generaron errores por la transferencia de imagenes

dirname = './images/11-09/lights/'
usigma, lsigma = 2,2
    #plt.figure(figsize=(12,10))
for filename in sorted(glob.glob(dirname+'*.fit')):
    try: 
        with fits.open(filename) as hdul:
            data = hdul[0].data # assume the first extension is an image
            header = hdul[0].header
        print(f'{filename} correct')
    except:
        print(f'{filename} causes error')

In [ ]:
#Checkeo simultáneo de imágenes de cada objecto, notar que cada filtro le corresponde un número según su posición en TSXPro en el PC@FALCON
#savedir = './pngs/dump/' #Uncoment to save
dirname = './images/11-09/lights/'
usigma, lsigma = 0.5, 0.5

for filename in sorted(glob.glob(dirname+'*.fit')):
    with fits.open(filename) as hdul:
        data = hdul[0].data # assume the first extension is an image
        header = hdul[0].header
    if header.get("FILTER") == None:
        colors = 'Greys'
    elif int(header.get("FILTER")) == 2: #B
        colors = 'Blues'
    elif int(header.get("FILTER")) == 3: #V
        colors = 'Greens'
    elif int(header.get("FILTER")) == 4: #R
        colors = 'Oranges'
    elif int(header.get("FILTER")) == 5: #I
        colors = 'Reds'
    elif int(header.get("FILTER")) == 11: #g'
        colors = 'Blues'
    elif int(header.get("FILTER")) == 12: #r'
        colors = 'Greens'
    elif int(header.get("FILTER")) == 13: #i'
        colors = 'Oranges'
    elif int(header.get("FILTER")) == 14: #z'
        colors = 'Reds'
    else:
        colors = 'Greys'
    datamedian = np.median(data)
    datastd = np.std(data)
    plt.figure(figsize=(12,10))
    plt.imshow(data, origin='lower', cmap=colors, vmin=np.mean(data)-lsigma*np.std(data), vmax=np.mean(data)+usigma*np.std(data))
    plt.tick_params(left = False, right = False, labelleft = False, labelbottom = False, bottom = False) 
    plt.title(filename.replace(dirname,''))
    plt.show
    #plt.savefig(f'{savedir}{filename.replace(dirname,"")}.png',format='png') #Uncomentt to save
    plt.close

## Calibrador de imágenes
Una vez revisadas las imagenes, procederé a crear los masters para la calibración de las imágenes \
Idealmente no correr más de una vez por carpeta ya que puede sobrescribir

In [ ]:
dirlight = './images/lights/'
dirdark = './images/darks/'
dirbias = './images/bias/'
dirflat1 = './images/flats/1/'
dirflat2 = './images/flats/2/'
dirflat4 = './images/flats/4/'
dirflat5 = './images/flats/5/'
dirflat12 = './images/flats/12/'
dirflat13 = './images/flats/13/'
out = './images/masters/'


In [ ]:
#Cargador de imágenes
def loader(dirname):
    images = []
    for file in sorted(glob.glob(dirname+'*.fit')): #Toma todos los fit de un (dir)ectorio, incluso si no son del mismo tipo
        with fits.open(file) as hdul:
            images.append(CCDData(hdul[0].data,unit=u.adu))
    print('Done',dirname)
    return images

bias = loader(dirbias)
#Discrimido por filtros en carpeta, donde blue es B, green es R y red es I (Jhonson)
flats1, flats12, flats13 = loader(dirflat1), loader(dirflat12), loader(dirflat13)
flatsblue, flatsgreen, flatsred = loader(dirflat2), loader(dirflat4), loader(dirflat5)

In [ ]:
def master_creator(images, method='median', scale=False):
    combined = ccdp.combine(images, method=method, sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5)
    if scale: #Si scale=True calcula escala las imagenes a mayores exptime
        combined = ccdp.subtract_bias(combined, master_creator(bias))
    print('Done')
    return combined

master_bias = master_creator(bias)
#Master por filtro
master_flat1 = master_creator(flats1)
master_flatblue = master_creator(flatsblue)
master_flatgreen = master_creator(flatsgreen)
master_flatred = master_creator(flatsred)
master_flat12 = master_creator(flats12)
master_flat13 = master_creator(flats13)

master_bias.write(os.path.join(out, 'master_bias.fits'), overwrite=True)
master_flat1.write(os.path.join(out, 'master_flat1.fits'), overwrite=True)
master_flatblue.write(os.path.join(out, 'master_flatblue.fits'), overwrite=True)
master_flatgreen.write(os.path.join(out, 'master_flatgreen.fits'), overwrite=True)
master_flatred.write(os.path.join(out, 'master_flatred.fits'), overwrite=True)
master_flat12.write(os.path.join(out, 'master_flat12.fits'), overwrite=True)
master_flat13.write(os.path.join(out, 'master_flat13.fits'), overwrite=True)

In [ ]:
#Fix darks por exptime
darks = loader(os.path.join(dirdark,'10/'))
master_dark = master_creator(darks)
master_dark.write(os.path.join(out, 'master_10dark.fits'), overwrite=True)

darks = loader(os.path.join(dirdark,'30/'))
master_dark = master_creator(darks)
master_dark.write(os.path.join(out, 'master_30dark.fits'), overwrite=True)

darks = loader(os.path.join(dirdark,'60/'))
master_dark = master_creator(darks)
master_dark.write(os.path.join(out, 'master_60dark.fits'), overwrite=True)

darks = loader(os.path.join(dirdark,'120/'))
master_dark = master_creator(darks)
master_dark.write(os.path.join(out, 'master_120dark.fits'), overwrite=True)

In [ ]:
#print(bias)

In [ ]:
#ccdp.subtract_bias(light, master_bias)
#ccdp.subtract_dark(calibrated, master_dark, exposure_time='EXPTIME', exposure_unit=u.second)
#ccdp.flat_correct(calibrated, master_flat)
#ccdmaster_bias = CCDData('./images/masters/master_bias.fits',unit=u.adu)

#CORRER PARA CADA CARPETA CON LIGHTS, NO CORRER MÁS DE UNA VEZ POR CARPETA
with fits.open('./images/masters/master_bias.fits') as hdul:
        master_bias = hdul[0].data.astype(float)

currentpath = './calibrations/'
for file in glob.glob(currentpath+'*.fit'):
    with fits.open(file) as hdul:
        data = hdul[0].data.astype(float)
        header = hdul[0].header
        #ccddata = CCDData(data, unit=u.adu)
    biased = (data - master_bias)
    fits.writeto(file, biased, header, overwrite=True)
    print('Done', file)

In [ ]:
#Por la falta de dark de 20 segundos, creo uno artificialmente
with fits.open('./images/masters/master_10dark.fits') as hdul:
    master_dark_10s = hdul[0].data.astype(float)
    
with fits.open('./images/masters/master_30dark.fits') as hdul:
    master_dark_30s = hdul[0].data.astype(float)

exposure_10s = 10
exposure_30s = 30
target_exposure = 20

weight_10s = (exposure_30s - target_exposure) / (exposure_30s - exposure_10s)
weight_30s = (target_exposure - exposure_10s) / (exposure_30s - exposure_10s)
master_dark_20s = weight_10s * master_dark_10s + weight_30s * master_dark_30s

output_filename = './images/masters/master_20dark.fits'
fits.writeto(output_filename, master_dark_20s, overwrite=True)

In [ ]:
def scale_dark(master_dark, target_exposure, dark_exposure):
    scale_factor = target_exposure / dark_exposure
    return master_dark * scale_factor

#CORRER PARA CADA CARPETA CON LIGHTS, NO CORRER MÁS DE UNA VEZ POR CARPETA
with fits.open('./images/masters/master_120dark.fits') as hdul:
        master_dark = hdul[0].data.astype(float)

scaled_dark = scale_dark(master_dark, target_exposure=150, dark_exposure=120)

currentpath = './images/lights/NGC 1812/'
for file in glob.glob(currentpath+'*.fit'):
    with fits.open(file) as hdul:
        data = hdul[0].data.astype(float)
        header = hdul[0].header
        #ccddata = CCDData(data, unit=u.adu)
    darkened = (data - scaled_dark)
    fits.writeto(file, darkened, header, overwrite=True)
    print('Done', file)

In [ ]:
#Thus, todas las imagenes están calibradas por bias y darks correspondientes, ahora con los flats
filter_mapping = {
    '5': 'red',
    '4': 'green',
    '2': 'blue'
}

def calibrate_flat(currentpath, masterpath):

    flat_frames = {}
    for flat_file in os.listdir(masterpath):
        if flat_file.endswith('.fits'):
            match = re.search(r'(red|green|blue)', flat_file, re.IGNORECASE)
            if match:
                color1 = match.group(1).lower()
                with fits.open(os.path.join(masterpath, flat_file)) as hdul:
                    flat_frames[color1] = hdul[0].data.astype(float) / np.median(hdul[0].data)

    for filename in os.listdir(currentpath):
        if filename.endswith('.fit'):
            match = re.search(r'-(\d{1,2})-', filename)
            if match:
                filter_number = match.group(1)
                color2 = filter_mapping.get(filter_number)
                
                if color1 and color2 in flat_frames:
                    with fits.open(os.path.join(currentpath,  filename)) as hdul:
                        image_data = hdul[0].data.astype(float)
                        header = hdul[0].header

                        calibrated_data = image_data / flat_frames[color1]
                        fits.writeto(filename, calibrated_data, header, overwrite=True)
                        print(f"Calibrated and saved: {filename}")
                else:
                    print(f"No matching flat frame for filter: {filter_number} in image {filename}")

masterpath = './images/masters/'

currentpath = './images/lights/NGC 1812/'
calibrate_flat(currentpath, masterpath)

currentpath = './images/lights/NGC 1976/'
calibrate_flat(currentpath, masterpath)

currentpath = './images/lights/NGC 2070/'
calibrate_flat(currentpath, masterpath)

currentpath = './images/lights/PCG 10488/'
calibrate_flat(currentpath, masterpath)

In [ ]:
#Checkeo de la Dark Current
currentpath = './images/masters/'
dark_frames = ['master_10dark.fits', 'master_30dark.fits', 'master_60dark.fits','master_120dark.fits']
exptimes = [10, 30, 60, 120]
means = []

for dark_frame in dark_frames:
    with fits.open(os.path.join(currentpath, dark_frame))as hdul:
        data = hdul[0].data
        mean_signal = np.mean(data)
        means.append(mean_signal)
plt.figure(figsize=(8,7))
plt.plot(exptimes, means, 'o-',color='purple')#, label='Dark Current')
plt.xlabel('Exposure Time (s)')
plt.ylabel('Mean Signal')
plt.title('Dark Current vs Exposure Time')
plt.grid(visible=True, which='major', axis='both')
plt.savefig('dqvsexptime.png')

In [ ]:
#Funciones para calcular Ganancia y Read Noise según Handbook of CCD Astronomy (fuente en ppt)

def gainfun(flat1, flat2, bias1, bias2):
    with fits.open(flat1) as hdul1, fits.open(flat2) as hdul2, fits.open(bias1) as hdul3, fits.open(bias2) as hdul4:
        data1 = hdul1[0].data.astype(float)
        data2 = hdul2[0].data.astype(float)
        data3 = hdul3[0].data.astype(float)
        data4 = hdul4[0].data.astype(float)

    mean1, mean2, mean3, mean4 = np.mean(data1), np.mean(data2), np.mean(data3), np.mean(data4)
    stdflat = np.std(data1-data2)
    stdbias = np.std(data3-data4)
    gain = ((mean1+mean2)-(mean3+mean4))/((stdflat**2)-(stdbias**2))
        
    #print(f"Calculated Gain: {round(gain, 4)} e-/ADU")
    return gain

def read_noise(bias1, bias2, gain): #Ahora tengo 1, pero sirve para más

    with fits.open(bias1) as hdul1, fits.open(bias2) as hdul2:
        data1 = hdul1[0].data.astype(float)
        data2 = hdul2[0].data.astype(float)

    stdbias = np.std(data1-data2)
    read_noise = (gain*stdbias)/(np.sqrt(2))
    
    #print(f"Calculated Read Noise: {round(read_noise, 4)} e-")
    return read_noise

In [ ]:
#Eligo los archivos y corro las funciones
flat1 = './images/flats/2/84-Entered Coordinates-2-0.2s-Flat Field.fit'
flat2 = './images/flats/5/88-Entered Coordinates-5-0.2s-Flat Field.fit'
bias1 = './images/bias/8-None-1-0.0s-Bias Frame.fit'
bias2 = './images/bias/9-None-1-0.0s-Bias Frame.fit'

gain = gainfun(flat1, flat2, bias1, bias2)
read_noise = read_noise(bias1, bias2, gain)

print(f"Calculated Gain: {round(gain, 4)} e-/ADU")
print(f"Calculated Read Noise: {round(read_noise, 4)} e-")

In [ ]:
#Según Handbook of CCD Astronomy estos flats debiesen ser mas gaussianos

file='./images/flats/2/84-Entered Coordinates-2-0.2s-Flat Field.fit'
with fits.open(file) as hdul:
    data=hdul[0].data
pixval = data.flatten()

plt.figure(figsize=(8,7))
plt.hist(pixval, bins=100, color='skyblue', edgecolor='black')
plt.xlabel('ADU')
plt.ylabel('Quantity')
plt.yscale('log')
plt.grid(True, linestyle='--', alpha=0.7)

meanblue=np.mean(pixval)

plt.savefig('blueflat.png')

In [ ]:
file='./images/flats/4/85-Entered Coordinates-4-0.2s-Flat Field.fit'
with fits.open(file) as hdul:
    data=hdul[0].data
pixval = data.flatten()

plt.figure(figsize=(8,7))
plt.hist(pixval, bins=100, color='limegreen', edgecolor='black')
plt.xlabel('ADU')
plt.ylabel('Quantity')
plt.yscale('log')
plt.grid(True, linestyle='--', alpha=0.7)

meangreen=np.mean(pixval)

plt.savefig('greenflat.png')

In [ ]:
file='./images/flats/5/88-Entered Coordinates-5-0.2s-Flat Field.fit'
with fits.open(file) as hdul:
    data=hdul[0].data
pixval = data.flatten()

plt.figure(figsize=(8,7))
plt.hist(pixval, bins=100, color='pink', edgecolor='black')
plt.xlabel('ADU')
plt.ylabel('Quantity')
plt.yscale('log')
plt.grid(True, linestyle='--', alpha=0.7)

meanred=np.mean(pixval)

plt.savefig('redflat.png')

In [ ]:
#Esta celda determina las siguientes 3 celdas
with fits.open('./fullcal/35-NGC 1976-4-6.0s-Light Frame.fit') as hdul:
    hdul.info()
    data = hdul[0].data.astype(float)
    header = hdul[0].header

In [ ]:
#3D histogram

y1, y2, x1, x2 = 3630, 3690, 4250, 4310
datasec = data[y1:y2,x1:x2]

x = np.repeat(np.arange(y1, y2), x2 - x1 )
y = np.tile(np.arange(x1, x2), y2 - y1)
pixel_values = datasec.flatten()

hist, edges = np.histogramdd((x, y, pixel_values), bins=(20, 20, 20))

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

xpos, ypos, zpos = np.meshgrid(
    (edges[0][:-1] + edges[0][1:]) / 2,
    (edges[1][:-1] + edges[1][1:]) / 2,
    (edges[2][:-1] + edges[2][1:]) / 2,
    indexing="ij")
xpos = xpos.ravel()
ypos = ypos.ravel()
zpos = zpos.flatten()
hist_values = hist.ravel()
width = depth = 1

nonzero = hist_values > 0
xpos, ypos, zpos, hist_values = xpos[nonzero], ypos[nonzero], zpos[nonzero], hist_values[nonzero]

ax.bar3d(xpos, ypos, zpos, 1, 1, 1, shade=True, color='Violet')
ax.set(xticklabels=[],
       yticklabels=[])
ax.set_ylabel('Columns')
ax.set_xlabel('Rows')
ax.set_zlabel('Pix Val')
ax.set_title('HD 37062')
ax.view_init(elev=15, azim=100)
plt.savefig('HD37062-3D.png')

In [ ]:
#2D histogram with marginals top/right

y1, y2, x1, x2 = 3630, 3690, 4250, 4310
datasec = data[y1:y2,x1:x2]

x = np.repeat(np.arange(y1, y2), x2 - x1 )
y = np.tile(np.arange(x1, x2), y2 - y1)
pixel_values = datasec.flatten()

fig = plt.figure(figsize=(12, 12))
gs = fig.add_gridspec(4, 3, hspace=0.05, wspace=0.05)

ax_main = fig.add_subplot(gs[1:, :-1])
hist, x_edges, y_edges, im = ax_main.hist2d(x, y, bins=60, weights=pixel_values, cmap='viridis')
fig.colorbar(im, ax=ax_main, label='ADUs',location='bottom')

ax_top = fig.add_subplot(gs[0, :-1], sharex=ax_main)
ax_top.hist(x, bins=60, weights=pixel_values+400, color='violet')
ax_top.tick_params(left = False, right = False, labelleft = False, labelbottom = False, bottom = False)

ax_right = fig.add_subplot(gs[1:3, -1], sharey=ax_main)
ax_right.hist(y, bins=60, weights=pixel_values+400, orientation='horizontal', color='violet')
ax_right.tick_params(left = False, right = False, labelleft = False, labelbottom = False, bottom = False)

# Set labels for the main plot
ax_main.set_xlabel('Rows')
ax_main.set_ylabel('Columns')

plt.savefig('HD37062-Hists.png')

In [ ]:
#Muestra del threshold de 250 negativas
threshold = -250
smoothed_data = gaussian_filter(data, sigma=1)

mask = smoothed_data >= threshold

fig = plt.figure(figsize=(12, 10))
plt.imshow(data, origin='lower', cmap='Greens', vmin=np.mean(data)-lsigma*np.std(data), vmax=np.mean(data)+usigma*np.std(data))
plt.title('"Calibrated" NGC 1976 @ R')

#Overlay
plt.contour(mask, colors='red', levels=[0.7], linewidths=0.1)

plt.tick_params(left = False, right = False, labelleft = False, labelbottom = False, bottom = False) 
plt.savefig('CalNGC1976R.png')